In [1]:
#| default_exp handlers.helcom

# HELCOM

> Data pipeline (handler) to convert HELCOM data ([source](https://helcom.fi/about-us)) to `NetCDF` format

<!-- ## HELCOM MORS Environment database -->

[Helcom MORS data](https://helcom.fi/about-us) is provided as a Microsoft Access database. 
[`Mdbtools`](https://github.com/mdbtools/mdbtools) can be used to convert the tables of the Microsoft Access database to `.csv` files on Unix-like OS.

Example steps:
1. Download data (e.g. https://metadata.helcom.fi/geonetwork/srv/fin/catalog.search#/metadata/2fdd2d46-0329-40e3-bf96-cb08c7206a24). 
2. Install mdbtools via VScode Terminal 

    ```
    sudo apt-get -y install mdbtools
    ````

3. Install unzip via VScode Terminal 

    ```
    sudo apt-get -y install unzip
    ````

4. In VS code terminal, navigate to the marisco data folder

    ```
    cd /home/marisco/downloads/marisco/_data/accdb/mors_19840101_20211231
    ```

5. Unzip MORS_ENVIRONMENT.zip 

    ```
    unzip MORS_ENVIRONMENT.zip 
    ```

6. Run preprocess.sh to generate the required data files

    ```
    ./preprocess.sh MORS_ENVIRONMENT.zip
    ````
7. Conetens of 'preprocess.sh' script.
    ```
    #!/bin/bash

    # Example of use: ./preprocess.sh MORS_ENVIRONMENT.zip
    unzip $1
    dbname=$(ls *.accdb)
    mkdir csv
    for table in $(mdb-tables -1 "$dbname"); do
        echo "Export table $table"
        mdb-export "$dbname" "$table" > "csv/$table.csv"
    done
    ```

***

## Packages import

In [2]:
%load_ext autoreload
%autoreload 2

In [217]:
#| export
import pandas as pd # Python package that provides fast, flexible, and expressive data structures.
import numpy as np
from tqdm import tqdm # Python Progress Bar Library
from functools import partial # Function which Return a new partial object which when called will behave like func called with the positional arguments args and keyword arguments keywords
import fastcore.all as fc # package that brings fastcore functionality, see https://fastcore.fast.ai/.
from pathlib import Path # This module offers classes representing filesystem paths
from dataclasses import asdict

from marisco.utils import (has_valid_varname, match_worms, match_maris_lut, Match)
from marisco.callbacks import (Callback, Transformer, EncodeTimeCB, SanitizeLonLatCB)
from marisco.metadata import (GlobAttrsFeeder, BboxCB, DepthRangeCB, TimeRangeCB, ZoteroCB, KeyValuePairCB)
from marisco.configs import (base_path, nc_tpl_path, cfg, cache_path, cdl_cfg, Enums, lut_path,
                             species_lut_path, sediments_lut_path, bodyparts_lut_path, 
                             detection_limit_lut_path, filtered_lut_path)
from marisco.serializers import NetCDFEncoder
from collections.abc import Callable

In [4]:
import warnings
warnings.filterwarnings('ignore')

Here we define the fname_in and fname_out variables. These variables are paths which are defined as relative paths. These paths are relative to 
the current working directory. Note that fname_in refers to the csv folder that contains the  HELCOM data. fname_out defines the path and filename for the NetCDF output.

In [5]:
# | export
fname_in = '../../_data/accdb/mors/csv'
fname_out = '../../_data/output/100-HELCOM-MORS-2024.nc'

***

## Utils

In [6]:
#| export
def load_data(src_dir,
                smp_types=['SEA', 'SED', 'BIO']):
    "Load HELCOM data and return the data in a dictionary of dataframes with the dictionary key as the sample type"
    dfs = {}
    lut_smp_type = {'SEA': 'seawater', 'SED': 'sediment', 'BIO': 'biota'}
    for smp_type in smp_types:
        fname_meas = smp_type + '02.csv' # measurement (i.e. radioactivity) information.
        fname_smp = smp_type + '01.csv' # sample information 
        df = pd.merge(pd.read_csv(Path(src_dir)/fname_meas),  # measurements
                      pd.read_csv(Path(src_dir)/fname_smp),  # sample
                      on='KEY', how='left')
        dfs[lut_smp_type[smp_type]] = df
    return dfs

In [7]:
#| export
def rename_cols(cols):
    "Flatten multiindex columns"
    new_cols = []
    for outer, inner in cols:
        if not inner:
            new_cols.append(outer)
        else:
            if outer == 'unit':
                new_cols.append(inner + '_' + outer)
            if outer == 'unc':
                new_cols.append(inner + '_' + outer)
            if outer == 'value':
                new_cols.append(inner)
    return new_cols

***

## Load tables

`dfs` is a dictionary dataframes  created from the Helcom dataset located at the path `fname_in`. The data to be included in each dataframe is sorted by sample type. Each dictionary is defined with a key equal to the sample type. 

In [8]:
#|eval: false
dfs = load_data(fname_in)
print(dfs.keys())
print(f"Seawater cols: {dfs['seawater'].columns}")
print(f"Sediment cols: {dfs['sediment'].columns}")
print(f"Biota cols: {dfs['biota'].columns}")

dict_keys(['seawater', 'sediment', 'biota'])
Seawater cols: Index(['KEY', 'NUCLIDE', 'METHOD', '< VALUE_Bq/m³', 'VALUE_Bq/m³', 'ERROR%_m³',
       'DATE_OF_ENTRY_x', 'COUNTRY', 'LABORATORY', 'SEQUENCE', 'DATE', 'YEAR',
       'MONTH', 'DAY', 'STATION', 'LATITUDE (ddmmmm)', 'LATITUDE (dddddd)',
       'LONGITUDE (ddmmmm)', 'LONGITUDE (dddddd)', 'TDEPTH', 'SDEPTH', 'SALIN',
       'TTEMP', 'FILT', 'MORS_SUBBASIN', 'HELCOM_SUBBASIN', 'DATE_OF_ENTRY_y'],
      dtype='object')
Sediment cols: Index(['KEY', 'NUCLIDE', 'METHOD', '< VALUE_Bq/kg', 'VALUE_Bq/kg', 'ERROR%_kg',
       '< VALUE_Bq/m²', 'VALUE_Bq/m²', 'ERROR%_m²', 'DATE_OF_ENTRY_x',
       'COUNTRY', 'LABORATORY', 'SEQUENCE', 'DATE', 'YEAR', 'MONTH', 'DAY',
       'STATION', 'LATITUDE (ddmmmm)', 'LATITUDE (dddddd)',
       'LONGITUDE (ddmmmm)', 'LONGITUDE (dddddd)', 'DEVICE', 'TDEPTH',
       'UPPSLI', 'LOWSLI', 'AREA', 'SEDI', 'OXIC', 'DW%', 'LOI%',
       'MORS_SUBBASIN', 'HELCOM_SUBBASIN', 'SUM_LINK', 'DATE_OF_ENTRY_y'],
      dty

Show the structure of the `biota` dataframe:

In [9]:
#|eval: false
dfs['biota'].head()

,KEY,NUCLIDE,METHOD,< VALUE_Bq/kg,VALUE_Bq/kg,BASIS,ERROR%,NUMBER,DATE_OF_ENTRY_x,COUNTRY,...,BIOTATYPE,TISSUE,NO,LENGTH,WEIGHT,DW%,LOI%,MORS_SUBBASIN,HELCOM_SUBBASIN,DATE_OF_ENTRY_y
0,BVTIG2012041,CS134,VTIG01,<,0.010140,W,NaN,NaN,02/27/14 00:00:00,6.0,...,F,5,16.0,45.7,948.0,18.453,92.9,2,16,02/27/14 00:00:00
1,BVTIG2012041,K40,VTIG01,,135.300000,W,3.57,NaN,02/27/14 00:00:00,6.0,...,F,5,16.0,45.7,948.0,18.453,92.9,2,16,02/27/14 00:00:00
2,BVTIG2012041,CO60,VTIG01,<,0.013980,W,NaN,NaN,02/27/14 00:00:00,6.0,...,F,5,16.0,45.7,948.0,18.453,92.9,2,16,02/27/14 00:00:00
3,BVTIG2012041,CS137,VTIG01,,4.338000,W,3.48,NaN,02/27/14 00:00:00,6.0,...,F,5,16.0,45.7,948.0,18.453,92.9,2,16,02/27/14 00:00:00
4,BVTIG2012040,CS134,VTIG01,<,0.009614,W,NaN,NaN,02/27/14 00:00:00,6.0,...,F,5,17.0,45.9,964.0,18.458,92.9,2,16,02/27/14 00:00:00


Show the structure of the `sediment` dataframe: 

In [10]:
#|eval: false
dfs['sediment'].head()

,KEY,NUCLIDE,METHOD,< VALUE_Bq/kg,VALUE_Bq/kg,ERROR%_kg,< VALUE_Bq/m²,VALUE_Bq/m²,ERROR%_m²,DATE_OF_ENTRY_x,...,LOWSLI,AREA,SEDI,OXIC,DW%,LOI%,MORS_SUBBASIN,HELCOM_SUBBASIN,SUM_LINK,DATE_OF_ENTRY_y
0,SKRIL2012048,RA226,NaN,NaN,35.0,26.0,NaN,NaN,NaN,08/20/14 00:00:00,...,20.0,0.006,NaN,NaN,NaN,NaN,11.0,11.0,NaN,08/20/14 00:00:00
1,SKRIL2012049,RA226,NaN,NaN,36.0,22.0,NaN,NaN,NaN,08/20/14 00:00:00,...,27.0,0.006,NaN,NaN,NaN,NaN,11.0,11.0,NaN,08/20/14 00:00:00
2,SKRIL2012050,RA226,NaN,NaN,38.0,24.0,NaN,NaN,NaN,08/20/14 00:00:00,...,2.0,0.006,NaN,NaN,NaN,NaN,11.0,11.0,NaN,08/20/14 00:00:00
3,SKRIL2012051,RA226,NaN,NaN,36.0,25.0,NaN,NaN,NaN,08/20/14 00:00:00,...,4.0,0.006,NaN,NaN,NaN,NaN,11.0,11.0,NaN,08/20/14 00:00:00
4,SKRIL2012052,RA226,NaN,NaN,30.0,23.0,NaN,NaN,NaN,08/20/14 00:00:00,...,6.0,0.006,NaN,NaN,NaN,NaN,11.0,11.0,NaN,08/20/14 00:00:00


***

## Data transformation pipeline

### Normalize nuclide names

#### Lower & strip

Create a callback `LowerStripRdnNameCB` that receives a dictionary of dataframes. For each dataframe in the dictionary of dataframes it converts the contents of the nuclide name column, `Nuclides`, to lowercase and strips any leading or trailing whitespace(s). 

In [11]:
#| export
class LowerStripRdnNameCB(Callback):
    "Convert nuclide names to lowercase & strip any trailing space(s)"
    def __call__(self, tfm):
        for k in tfm.dfs.keys():
            tfm.dfs[k]['NUCLIDE'] = tfm.dfs[k]['NUCLIDE'].apply(
                lambda x: x.lower().strip())

Here we apply the transform `LowerStripRdnNameCB`. Then print the nuclide names that are unique from the column, `NUCLIDE`, for each dataframe include in the dictionary of dataframes, dfs.

In [12]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB()])
print('seawater nuclides: ')
print(tfm()['seawater']['NUCLIDE'].unique())
print('biota nuclides: ')
print(tfm()['biota']['NUCLIDE'].unique())
print('sediment nuclides: ')
print(tfm()['sediment']['NUCLIDE'].unique())

seawater nuclides: 
['cs137' 'sr90' 'h3' 'cs134' 'pu238' 'pu239240' 'am241' 'cm242' 'cm244'
 'tc99' 'k40' 'ru103' 'sr89' 'sb125' 'nb95' 'ru106' 'zr95' 'ag110m'
 'cm243244' 'ba140' 'ce144' 'u234' 'u238' 'co60' 'pu239' 'pb210' 'po210'
 'np237' 'pu240' 'mn54']
biota nuclides: 
['cs134' 'k40' 'co60' 'cs137' 'sr90' 'ag108m' 'mn54' 'co58' 'ag110m'
 'zn65' 'sb125' 'pu239240' 'ru106' 'be7' 'ce144' 'pb210' 'po210' 'sb124'
 'sr89' 'zr95' 'te129m' 'ru103' 'nb95' 'ce141' 'la140' 'i131' 'ba140'
 'pu238' 'u235' 'bi214' 'pb214' 'pb212' 'tl208' 'ac228' 'ra223' 'eu155'
 'ra226' 'gd153' 'sn113' 'fe59' 'tc99' 'co57' 'sn117m' 'eu152' 'sc46'
 'rb86' 'ra224' 'th232' 'cs134137' 'am241' 'ra228' 'th228' 'k-40' 'cs138'
 'cs139' 'cs140' 'cs141' 'cs142' 'cs143' 'cs144' 'cs145' 'cs146']
sediment nuclides: 
['ra226' 'cs137' 'ra228' 'k40' 'sr90' 'cs134137' 'cs134' 'pu239240'
 'pu238' 'co60' 'ru103' 'ru106' 'sb125' 'ag110m' 'ce144' 'am241' 'be7'
 'th228' 'pb210' 'co58' 'mn54' 'zr95' 'ba140' 'po210' 'ra224' 'nb95'
 'p

#### Remap to MARIS nuclide names 

In [13]:
#| export
def get_unique_nuclides(dfs):
    "Get list of unique radionuclide types measured across samples."
    nuclides = []
    for k in dfs.keys():
        nuclides += dfs[k]['NUCLIDE'].unique().tolist()
    return nuclides

The marisco package includes a template that defines the permitted structure of the data. This includes a list of permitted nuclide names. This template is located at `nc_tpl_path` and is a `NetCDF` file ,`*.nc` format.
This template is created using [`nc_template.ipynb` notebook](../api/nc_template.ipynb). The template is created with reference to two files, `cdl.toml` and `configs.toml`. The `cdl.toml` file includes all the permitted nuclide names. 

Here we check if the nuclide names included in the dfs are listed in the template that defines the permitted structures:

In [14]:
#|eval: false
# Check if these variable names consistent with MARIS CDL
has_valid_varname(get_unique_nuclides(tfm.dfs), nc_tpl_path())

"pu239240" variable name not found in MARIS CDL
"cm243244" variable name not found in MARIS CDL
"cs134137" variable name not found in MARIS CDL
"pu239240" variable name not found in MARIS CDL
"pu238240" variable name not found in MARIS CDL
"pu239240" variable name not found in MARIS CDL
"cs134137" variable name not found in MARIS CDL
"k-40" variable name not found in MARIS CDL
"cs138" variable name not found in MARIS CDL
"cs139" variable name not found in MARIS CDL
"cs140" variable name not found in MARIS CDL
"cs141" variable name not found in MARIS CDL
"cs142" variable name not found in MARIS CDL
"cs143" variable name not found in MARIS CDL
"cs144" variable name not found in MARIS CDL
"cs145" variable name not found in MARIS CDL
"cs146" variable name not found in MARIS CDL


False

Many nuclide names are not listed in the MARIS CDL (i.e. `cdl.toml`).
Create a look up table, `varnames_lut_updates`, which will be used to correct the nuclide names in the dictionary of dataframes (i.e. dfs) that are not compatible with the template at `nc_tpl_path` 

In [15]:
#| export
varnames_lut_updates = {
    'k-40': 'k40',
    'cm243244': 'cm243_244_tot',
    'cs134137': 'cs134_137_tot',
    'pu239240': 'pu239_240_tot',
    'pu238240': 'pu238_240_tot',
    'cs138': 'cs137',
    'cs139': 'cs137',
    'cs140': 'cs137',
    'cs141': 'cs137',
    'cs142': 'cs137',
    'cs143': 'cs137',
    'cs144': 'cs137',
    'cs145': 'cs137',
    'cs146': 'cs137'}

Create a function, `get_varnames_lut`, which returns a dictionary of nuclide names. This dictionary of nuclide names includes the `NUCLIDE` names in the dictionary of dataframes (i.e. dfs) with the corrections included in varnames_lut_updates:

In [16]:
#| export
def get_varnames_lut(dfs, lut=varnames_lut_updates):
    lut = {n: n for n in set(get_unique_nuclides(dfs))}
    lut.update(varnames_lut_updates)
    return lut

Create a callback that remaps the nuclide names in the dfs to those in `varnames_lut_updates`:

In [17]:
# | export
class RemapRdnNameCB(Callback):
    "Remap to MARIS radionuclide names."
    def __init__(self,
                 fn_lut=partial(get_varnames_lut, lut=varnames_lut_updates)):
        fc.store_attr()

    def __call__(self, tfm):
        lut = self.fn_lut(tfm.dfs)
        for k in tfm.dfs.keys():
            tfm.dfs[k]['NUCLIDE'].replace(lut, inplace=True)

Apply the transformers `LowerStripRdnNameCB` and `RemapRdnNameCB`. Print the unique nuclides for each dataframe included in the dictionary of dataframes:

In [18]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB()])

print('seawater nuclides: ')
print(tfm()['seawater']['NUCLIDE'].unique())
print('biota nuclides: ')
print(tfm()['biota']['NUCLIDE'].unique())
print('sediment nuclides: ')
print(tfm()['sediment']['NUCLIDE'].unique())

seawater nuclides: 
['cs137' 'sr90' 'h3' 'cs134' 'pu238' 'pu239_240_tot' 'am241' 'cm242'
 'cm244' 'tc99' 'k40' 'ru103' 'sr89' 'sb125' 'nb95' 'ru106' 'zr95'
 'ag110m' 'cm243_244_tot' 'ba140' 'ce144' 'u234' 'u238' 'co60' 'pu239'
 'pb210' 'po210' 'np237' 'pu240' 'mn54']
biota nuclides: 
['cs134' 'k40' 'co60' 'cs137' 'sr90' 'ag108m' 'mn54' 'co58' 'ag110m'
 'zn65' 'sb125' 'pu239_240_tot' 'ru106' 'be7' 'ce144' 'pb210' 'po210'
 'sb124' 'sr89' 'zr95' 'te129m' 'ru103' 'nb95' 'ce141' 'la140' 'i131'
 'ba140' 'pu238' 'u235' 'bi214' 'pb214' 'pb212' 'tl208' 'ac228' 'ra223'
 'eu155' 'ra226' 'gd153' 'sn113' 'fe59' 'tc99' 'co57' 'sn117m' 'eu152'
 'sc46' 'rb86' 'ra224' 'th232' 'cs134_137_tot' 'am241' 'ra228' 'th228']
sediment nuclides: 
['ra226' 'cs137' 'ra228' 'k40' 'sr90' 'cs134_137_tot' 'cs134'
 'pu239_240_tot' 'pu238' 'co60' 'ru103' 'ru106' 'sb125' 'ag110m' 'ce144'
 'am241' 'be7' 'th228' 'pb210' 'co58' 'mn54' 'zr95' 'ba140' 'po210'
 'ra224' 'nb95' 'pu238_240_tot' 'pu241' 'pu239' 'eu155' 'ir192' 'th2

Check that all nuclide varnames are valid. Returns `True` if all are valid:

In [19]:
#|eval: false
has_valid_varname(get_unique_nuclides(tfm.dfs), nc_tpl_path())

True

***

### Parse time

Create a class that remaps the time format in the dictionary of dataframes (i.e. `%m/%d/%y %H:%M:%S`):

In [20]:
#| export
class ParseTimeCB(Callback):
    def __call__(self, tfm):
        for k in tfm.dfs.keys():
            tfm.dfs[k]['time'] = pd.to_datetime(tfm.dfs[k].DATE, 
                                                format='%m/%d/%y %H:%M:%S')

In [21]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB()])

print(tfm()['seawater']['time'][:5])

0   2012-05-23
1   2012-05-23
2   2012-06-17
3   2012-05-24
4   2012-05-24
Name: time, dtype: datetime64[ns]


***

### Normalize uncertainty units

In [22]:
#| export
# Make measurement and uncertainty units consistent
def fix_units(df, meas_col, unc_col):
    return df.apply(lambda row: row[unc_col] * row[meas_col]/100, axis=1)

For each sample type of the Helcom dataset the uncertainty is provided as a relative uncertainty to the value. The column names for each sample type differ.  The `coi_units_unc` defines the column name of the Value and Uncertainty for each sample type.

In [23]:
#| export
# Columns of interest
coi_units_unc = [('seawater', 'VALUE_Bq/m³', 'ERROR%_m³'),
                 ('biota', 'VALUE_Bq/kg', 'ERROR%'),
                 ('sediment', 'VALUE_Bq/kg', 'ERROR%_kg')]

Normalize the uncertainty. The relative error is converted to the uncertainty with the same units as the value. 

In [24]:
#| export
class NormalizeUncUnitCB(Callback):
    "Convert from relative error % to uncertainty of activity unit"
    def __init__(self, coi=coi_units_unc): fc.store_attr()

    def __call__(self, tfm):
        for grp, val, unc in self.coi:
            tfm.dfs[grp][unc] = self.fix_units(tfm.dfs[grp], val, unc)

    def fix_units(self, df, meas_col, unc_col):
        return df.apply(lambda row: row[unc_col] * row[meas_col]/100, axis=1)

In [25]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            NormalizeUncUnitCB()])

print(tfm()['seawater'][['VALUE_Bq/m³', 'ERROR%_m³']][:5])
print(tfm()['biota'][['VALUE_Bq/kg', 'ERROR%']][:5])
print(tfm()['sediment'][['VALUE_Bq/kg', 'ERROR%_kg']][:5])

   VALUE_Bq/m³  ERROR%_m³
0          5.3      1.696
1         19.9      3.980
2         25.5      5.100
3         17.0      4.930
4         22.2      3.996
   VALUE_Bq/kg    ERROR%
0     0.010140       NaN
1   135.300000  6.535274
2     0.013980       NaN
3     4.338000  0.006549
4     0.009614       NaN
   VALUE_Bq/kg  ERROR%_kg
0         35.0   1.114750
1         36.0   1.026432
2         38.0   1.316928
3         36.0   1.166400
4         30.0   0.621000


***

### Lookup : Biota species

Get a list of rubin names included in the Helcom data i.e.( `RUBIN_NAME.csv`):

In [26]:
#|eval: false
df_rubin = pd.read_csv(Path(fname_in) / 'RUBIN_NAME.csv')
df_rubin.head(5)

,RUBIN_ID,RUBIN,SCIENTIFIC NAME,ENGLISH NAME
0,11,ABRA BRA,ABRAMIS BRAMA,BREAM
1,12,ANGU ANG,ANGUILLA ANGUILLA,EEL
2,13,ARCT ISL,ARCTICA ISLANDICA,ISLAND CYPRINE
3,14,ASTE RUB,ASTERIAS RUBENS,COMMON STARFISH
4,15,CARD EDU,CARDIUM EDULE,COCKLE


In [27]:
#|eval: false
df_rubin[['RUBIN', 'SCIENTIFIC NAME']].drop_duplicates().head()

,RUBIN,SCIENTIFIC NAME
0,ABRA BRA,ABRAMIS BRAMA
1,ANGU ANG,ANGUILLA ANGUILLA
2,ARCT ISL,ARCTICA ISLANDICA
3,ASTE RUB,ASTERIAS RUBENS
4,CARD EDU,CARDIUM EDULE


In [28]:
#|export
def get_maris_lut(fname_in, 
                  fname_cache, # For instance 'species_helcom.pkl'
                  data_provider_lut:str, # Data provider lookup table name
                  data_provider_id_col:str, # Data provider lookup column id of interest
                  data_provider_name_col:str, # Data provider lookup column name of interest
                  maris_lut:Callable, # Function retrieving MARIS source lookup table
                  maris_id: str, # Id of MARIS lookup table nomenclature item to match
                  maris_name: str, # Name of MARIS lookup table nomenclature item to match
                  unmatched_fixes={},
                  as_dataframe=False,
                  overwrite=False
                 ):
    fname_cache = cache_path() / fname_cache
    lut = {}
    maris_lut = maris_lut()
    df = pd.read_csv(Path(fname_in) / data_provider_lut)
    if overwrite or (not fname_cache.exists()):
        for _, row in tqdm(df.iterrows(), total=len(df)):

            # Fix if unmatched
            has_to_be_fixed = row[data_provider_id_col] in unmatched_fixes            
            name_to_match = unmatched_fixes[row[data_provider_id_col]] if has_to_be_fixed else row[data_provider_name_col]

            # Match
            result = match_maris_lut(maris_lut, name_to_match, maris_id, maris_name)
            match = Match(result.iloc[0][maris_id], result.iloc[0][maris_name], 
                          row[data_provider_name_col], result.iloc[0]['score'])
            
            lut[row[data_provider_id_col]] = match
        fc.save_pickle(fname_cache, lut)
    else:
        lut = fc.load_pickle(fname_cache)

    if as_dataframe:
        df_lut = pd.DataFrame({k: asdict(v) for k, v in lut.items()}).transpose()
        df_lut.index.name = 'source_id'
        return df_lut.sort_values(by='match_score', ascending=False)
    else:
        return lut

In [29]:
#|export
unmatched_fixes_biota_species = {
    'CARD EDU': 'Cerastoderma edule',
    'LAMI SAC': 'Saccharina latissima',
    'PSET MAX': 'Scophthalmus maximus',
    'STIZ LUC': 'Sander luciopercas'}

In [30]:
#|eval: false
species_lut_df = get_maris_lut(fname_in, fname_cache='species_helcom.pkl', 
                               data_provider_lut='RUBIN_NAME.csv',
                               data_provider_id_col='RUBIN',
                               data_provider_name_col='SCIENTIFIC NAME',
                               maris_lut=species_lut_path,
                               maris_id='species_id',
                               maris_name='species',
                               unmatched_fixes=unmatched_fixes_biota_species,
                               as_dataframe=True,
                               overwrite=True)

  0%|          | 0/43 [00:00<?, ?it/s]

100%|██████████| 43/43 [00:07<00:00,  5.48it/s]


Show `species_lut` as a dataframe:

In [31]:
#|eval: false
species_lut_df

,matched_id,matched_maris_name,source_name,match_score
source_id,,,,
ENCH CIM,276,Echinodermata,ENCHINODERMATA CIM,5
MACO BAL,122,Macoma balthica,MACOMA BALTICA,1
STUC PEC,704,Stuckenia pectinata,STUCKENIA PECTINATE,1
STIZ LUC,285,Sander lucioperca,STIZOSTEDION LUCIOPERCA,1
POLY FUC,245,Polysiphonia fucoides,POLYSIPHONIA FUCOIDES,0
MYOX SCO,278,Myoxocephalus scorpius,MYOXOCEPHALUS SCORPIUS,0
MYTI EDU,129,Mytilus edulis,MYTILUS EDULIS,0
OSME EPE,279,Osmerus eperlanus,OSMERUS EPERLANUS,0
PERC FLU,247,Perca fluviatilis,PERCA FLUVIATILIS,0


Show maris_species_lut where match_type is not a perfect match ( i.e. not equal 0).

In [32]:
species_lut_df[species_lut_df['match_score'] > 1]

,matched_id,matched_maris_name,source_name,match_score
source_id,,,,
ENCH CIM,276,Echinodermata,ENCHINODERMATA CIM,5


In [33]:
#| export
class LookupBiotaSpeciesCB(Callback):
    """
    Biota species remapped to MARIS db:
        CARD EDU: Cerastoderma edule
        LAMI SAC: Saccharina latissima
        PSET MAX: Scophthalmus maximus
        STIZ LUC: Sander luciopercas
    """
    def __init__(self, fn_lut): fc.store_attr()
    def __call__(self, tfm):
        lut = self.fn_lut()
        tfm.dfs['biota']['species'] = tfm.dfs['biota']['RUBIN'].apply(lambda x: lut[x.strip()].matched_id)

In [34]:
#| export
get_maris_species = partial(get_maris_lut,
                            fname_in, fname_cache='species_helcom.pkl', 
                            data_provider_lut='RUBIN_NAME.csv',
                            data_provider_id_col='RUBIN',
                            data_provider_name_col='SCIENTIFIC NAME',
                            maris_lut=species_lut_path,
                            maris_id='species_id',
                            maris_name='species',
                            unmatched_fixes=unmatched_fixes_biota_species,
                            as_dataframe=False,
                            overwrite=False)

In [35]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            LookupBiotaSpeciesCB(get_maris_species)
                            ])

#print(tfm()['biota'][['RUBIN', 'species']][:10])
print(tfm()['biota']['species'].unique())

[  99  243   50  139  270  192  191  284   84  269  122   96  287  279
  278  288  286  244  129  275  271  285  283  247  120   59  280  274
  273  290  289  272  277  276   21  282  110  281  245  704 1524]


***

### Lookup : Biota tissues

In [36]:
#|eval: false
pd.read_csv('../../_data/accdb/mors/csv/TISSUE.csv')

,TISSUE,TISSUE_DESCRIPTION
0,1,WHOLE FISH
1,2,WHOLE FISH WITHOUT ENTRAILS
2,3,WHOLE FISH WITHOUT HEAD AND ENTRAILS
3,4,FLESH WITH BONES
4,5,FLESH WITHOUT BONES (FILETS)
5,6,HEAD
6,7,FINS
7,8,SKIN/EPIDERMIS
8,9,SCALES
9,10,BONES


In [37]:
#|export
unmatched_fixes_biota_tissues = {
    3: 'Whole animal eviscerated without head',
    12: 'Viscera',
    8: 'Skin'}

In [38]:
#|eval: false
tissues_lut_df = get_maris_lut(fname_in, 
                               fname_cache='tissues_helcom.pkl', 
                               data_provider_lut='TISSUE.csv',
                               data_provider_id_col='TISSUE',
                               data_provider_name_col='TISSUE_DESCRIPTION',
                               maris_lut=bodyparts_lut_path,
                               maris_id='bodypar_id',
                               maris_name='bodypar',
                               unmatched_fixes=unmatched_fixes_biota_tissues,
                               as_dataframe=True,
                               overwrite=True)

100%|██████████| 29/29 [00:00<00:00, 102.77it/s]


In [39]:
tissues_lut_df

,matched_id,matched_maris_name,source_name,match_score
source_id,,,,
2,52,Flesh without bones,WHOLE FISH WITHOUT ENTRAILS,13
5,52,Flesh without bones,FLESH WITHOUT BONES (FILETS),9
1,1,Whole animal,WHOLE FISH,5
15,53,Stomach and intestine,STOMACH + INTESTINE,3
41,1,Whole animal,WHOLE ANIMALS,1
4,4,Flesh with bones,FLESH WITH BONES,0
18,25,Liver,LIVER,0
54,57,Upper parts of plants,UPPER PARTS OF PLANTS,0
53,56,Growing tips,GROWING TIPS,0


In [40]:
#| export
class LookupBiotaBodyPartCB(Callback):
    """
    Update bodypart id based on MARIS dbo_bodypar.xlsx:
        - 3: 'Whole animal eviscerated without head',
        - 12: 'Viscera',
        - 8: 'Skin'
    """
    def __init__(self, fn_lut): fc.store_attr()
    def __call__(self, tfm):
        lut = self.fn_lut()
        tfm.dfs['biota']['body_part'] = tfm.dfs['biota']['TISSUE'].apply(lambda x: lut[x].matched_id)

In [41]:
#|eval: false
tissues_lut_df = get_maris_lut(fname_in, 
                               fname_cache='tissues_helcom.pkl', 
                               data_provider_lut='TISSUE.csv',
                               data_provider_id_col='TISSUE',
                               data_provider_name_col='TISSUE_DESCRIPTION',
                               maris_lut=bodyparts_lut_path,
                               maris_id='bodypar_id',
                               maris_name='bodypar',
                               unmatched_fixes=unmatched_fixes_biota_tissues,
                               as_dataframe=True,
                               overwrite=True)

100%|██████████| 29/29 [00:00<00:00, 102.06it/s]


In [42]:
#| export
get_maris_bodypart = partial(get_maris_lut,
                             fname_in,
                             fname_cache='tissues_helcom.pkl', 
                             data_provider_lut='TISSUE.csv',
                             data_provider_id_col='TISSUE',
                             data_provider_name_col='TISSUE_DESCRIPTION',
                             maris_lut=bodyparts_lut_path,
                             maris_id='bodypar_id',
                             maris_name='bodypar',
                             unmatched_fixes=unmatched_fixes_biota_tissues)

In [43]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            LookupBiotaBodyPartCB(get_maris_bodypart)
                            ])

print(tfm()['biota'][['TISSUE', 'body_part']][:5])

   TISSUE  body_part
0       5         52
1       5         52
2       5         52
3       5         52
4       5         52


***

### Lookup : Sediment types

In [44]:
#|eval: false
df_sediment = pd.read_csv(Path(fname_in) / 'SEDIMENT_TYPE.csv')
df_sediment

,SEDI,SEDIMENT TYPE,RECOMMENDED TO BE USED
0,-99,NO DATA,NaN
1,30,SILT AND GRAVEL,YES
2,0,GRAVEL,YES
3,1,SAND,YES
4,2,FINE SAND,NO
5,3,SILT,YES
6,4,CLAY,YES
7,5,MUD,YES
8,6,GLACIAL,NO
9,7,SOFT,NO


In [45]:
#|export
unmatched_fixes_sediments = {
    #np.nan: 'Not applicable',
    -99: '(Not available)'
}

In [46]:
#|eval: false
sediments_lut_df = get_maris_lut(
    fname_in, 
    fname_cache='sediments_helcom.pkl', 
    data_provider_lut='SEDIMENT_TYPE.csv',
    data_provider_id_col='SEDI',
    data_provider_name_col='SEDIMENT TYPE',
    maris_lut=sediments_lut_path,
    maris_id='sedtype_id',
    maris_name='sedtype',
    unmatched_fixes=unmatched_fixes_sediments,
    as_dataframe=True,
    overwrite=True)

100%|██████████| 47/47 [00:00<00:00, 86.37it/s]


In [48]:
#| export
get_maris_sediments = partial(
    get_maris_lut,
    fname_in, 
    fname_cache='sediments_helcom.pkl', 
    data_provider_lut='SEDIMENT_TYPE.csv',
    data_provider_id_col='SEDI',
    data_provider_name_col='SEDIMENT TYPE',
    maris_lut=sediments_lut_path,
    maris_id='sedtype_id',
    maris_name='sedtype',
    unmatched_fixes=unmatched_fixes_sediments)

In [49]:
#| export
class LookupSedimentCB(Callback):
    """
    Update sediment id  based on MARIS dbo_sedtype.xlsx
        -99: '(Not available)'
        - na: '(Not available)'
        - 56: '(Not available)'
        - 73: '(Not available)'
    """
    def __init__(self, fn_lut): fc.store_attr()
    def __call__(self, tfm):
        lut = self.fn_lut()

        # To check with Helcom
        tfm.dfs['sediment']['SEDI'] = dfs['sediment']['SEDI'].fillna(-99).astype('int')
        tfm.dfs['sediment']['SEDI'].replace(56, -99, inplace=True)
        tfm.dfs['sediment']['SEDI'].replace(73, -99, inplace=True)
        
        tfm.dfs['sediment']['sed_type'] = tfm.dfs['sediment']['SEDI'].apply(lambda x: lut[x].matched_id)

In [50]:
#|eval: false
dfs['sediment']['SEDI'].unique()

array([ nan, -99.,   0.,  55.,  11.,  57.,  51.,  52.,  22.,  10.,  44.,
         5.,  50.,  15.,   1.,  40.,  33.,  43.,  59.,  54.,   9.,  45.,
        14.,  41.,  25.,  42.,  24.,  12.,  58.,  13.,   7.,  49.,  48.,
         4.,  47.,  23.,  20.,  46.,   2.,  34.,  32.,  56.,  35.,  73.,
        21.])

In [51]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            LookupSedimentCB(get_maris_sediments)
                            ])

print(tfm()['sediment'][['SEDI', 'sed_type']][:5])
print(tfm())

   SEDI  sed_type
0   -99         0
1   -99         0
2   -99         0
3   -99         0
4   -99         0
{'seawater':                 KEY NUCLIDE  METHOD < VALUE_Bq/m³  VALUE_Bq/m³  ERROR%_m³  \
0      WKRIL2012003   cs137     NaN           NaN          5.3       32.0   
1      WKRIL2012004   cs137     NaN           NaN         19.9       20.0   
2      WKRIL2012005   cs137     NaN           NaN         25.5       20.0   
3      WKRIL2012006   cs137     NaN           NaN         17.0       29.0   
4      WKRIL2012007   cs137     NaN           NaN         22.2       18.0   
...             ...     ...     ...           ...          ...        ...   
20313  WDHIG2015227    sr90  DHIG02           NaN          6.6        7.5   
20314  WDHIG2015237    sr90  DHIG02           NaN          6.9        7.5   
20315  WDHIG2015239    sr90  DHIG02           NaN          6.8        7.5   
20316  WDHIG2015255    sr90  DHIG02           NaN          7.3        7.5   
20317  WDHIG2015256    sr90  DHI

***

### Lookup : Biogoup

'Biogroup' is not in the OpenRefine csv format. Should we drop from handler? 

In [53]:
'''
#| export
def get_biogroup_lut(maris_lut):
    species = pd.read_excel(maris_lut)
    return species[['species_id', 'biogroup_id']].set_index('species_id').to_dict()['biogroup_id']
'''

"\n#| export\ndef get_biogroup_lut(maris_lut):\n    species = pd.read_excel(maris_lut)\n    return species[['species_id', 'biogroup_id']].set_index('species_id').to_dict()['biogroup_id']\n"

In [54]:
'''
#| export
class LookupBiogroupCB(Callback):
    """
    Update biogroup id  based on MARIS dbo_species.xlsx
    """
    def __init__(self, fn_lut): fc.store_attr()
    def __call__(self, tfm):
        lut = self.fn_lut()
        tfm.dfs['biota']['bio_group'] = tfm.dfs['biota']['species'].apply(lambda x: lut[x])
'''

'\n#| export\nclass LookupBiogroupCB(Callback):\n    """\n    Update biogroup id  based on MARIS dbo_species.xlsx\n    """\n    def __init__(self, fn_lut): fc.store_attr()\n    def __call__(self, tfm):\n        lut = self.fn_lut()\n        tfm.dfs[\'biota\'][\'bio_group\'] = tfm.dfs[\'biota\'][\'species\'].apply(lambda x: lut[x])\n'

In [55]:
'''
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path()))
                            ])

print(tfm()['biota']['bio_group'].unique())
'''

"\n#|eval: false\ndfs = load_data(fname_in)\ntfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),\n                            RemapRdnNameCB(),\n                            ParseTimeCB(),\n                            LookupBiotaSpeciesCB(get_maris_species),\n                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path()))\n                            ])\n\nprint(tfm()['biota']['bio_group'].unique())\n"

***

### Lookup : Units

In [56]:
#| export
# Define unit names renaming rules
renaming_unit_rules = {'VALUE_Bq/m³': 1, #'Bq/m3'
                       'VALUE_Bq/kg': 3 } #'Bq/kg'

In [111]:
#| export
class LookupUnitCB(Callback):
    def __init__(self,
                 renaming_unit_rules=renaming_unit_rules):
        fc.store_attr()
    def __call__(self, tfm):
        for grp in tfm.dfs.keys():
            for k,v in self.renaming_unit_rules.items():
                if k in tfm.dfs[grp].columns:
                    tfm.dfs[grp]['unit'] = np.where(tfm.dfs[grp].loc[:,k].notna(), np.int64(v), np.int64(0))


In [112]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            NormalizeUncUnitCB(),
                            LookupUnitCB()])

print(tfm()['seawater'].unit.unique())

[1 0]


***

### Lookup : Value type (_dl) 

In [166]:
#| export
# Columns of interest
coi_dl = {'seawater' : { 'val' : 'VALUE_Bq/m³',
                        'unc' : 'ERROR%_m³',
                        'dl' : '< VALUE_Bq/m³'},
                 'biota':  {'val' : 'VALUE_Bq/kg',
                            'unc' : 'ERROR%',
                            'dl' : '< VALUE_Bq/kg'},
                 'sediment': { 'val' : 'VALUE_Bq/kg',
                              'unc' : 'ERROR%_kg',
                              'dl' : '< VALUE_Bq/kg'}}

In [218]:
#| export 
def get_detectionlimitnames_lut():
    df = pd.read_excel(detection_limit_lut_path(), usecols=['name','id'])
    return df.set_index('name').to_dict()['id']

In [219]:
# | export
class LookupDetectionLimitCB(Callback):
    "Remamp value type to MARIS format."
    def __init__(self ,
                 coi=coi_dl,
                 fn_lut=get_detectionlimitnames_lut
                 ):
        fc.store_attr()

    def __call__(self, tfm):
        lut = self.fn_lut()
        for grp in tfm.dfs.keys():
            # Copy val_t col 
            tfm.dfs[grp]['detection_limit'] = tfm.dfs[grp][self.coi[grp]['dl']]
            # Fill values with '=' if both a value and uncertainty are not nan and detection_limit is not in the list of keys returned from lut.
            condition = ((tfm.dfs[grp][self.coi[grp]['val']].notna()) | (tfm.dfs[grp][self.coi[grp]['unc']].notna())) & (~tfm.dfs[grp]["detection_limit"].isin(list(lut.keys())))
            tfm.dfs[grp].loc[condition, 'detection_limit']= '='
            # Fill values that are not in the lut with 'Not Available'.
            tfm.dfs[grp].loc[~tfm.dfs[grp]["detection_limit"].isin(list(lut.keys())), "detection_limit"] = 'Not Available'
            # Perform lookup
            tfm.dfs[grp]['detection_limit'] = tfm.dfs[grp]['detection_limit'].apply(lambda x: lut[x])
            

In [220]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            NormalizeUncUnitCB(),
                            LookupUnitCB(),
                            LookupDetectionLimitCB()])

print(tfm()['seawater']['detection_limit'].unique())

[1 2 0]


***

### Lookup : METHOD 

TODO: 'Method' provided in Helcom but some work required to link to Maris 'counting_method', 'sampling_method' and 'preparation_method'.

In [207]:
tfm.dfs['seawater']['KEY']

0        WKRIL2012003
1        WKRIL2012004
2        WKRIL2012005
3        WKRIL2012006
4        WKRIL2012007
             ...     
20313    WDHIG2015227
20314    WDHIG2015237
20315    WDHIG2015239
20316    WDHIG2015255
20317    WDHIG2015256
Name: KEY, Length: 20318, dtype: object

***

### Data provider sample id

In [209]:
# | export
class RemampDataProviderSampleIdCB(Callback):
    "Remamp key to MARIS data_provider_sample_id format."
    def __init__(self):
        fc.store_attr()

    def __call__(self, tfm):
        for grp in tfm.dfs.keys():
            # data_provider_sample_id
            tfm.dfs[grp]['data_provider_sample_id'] = tfm.dfs[grp]['KEY']

In [212]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            NormalizeUncUnitCB(),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),
                            RemampDataProviderSampleIdCB()])

print(tfm()['seawater']['data_provider_sample_id'].unique())

['WKRIL2012003' 'WKRIL2012004' 'WKRIL2012005' ... 'WSSSM2018006'
 'WSSSM2018007' 'WSSSM2018008']


***

### Lookup : FILT 

In [ ]:
#| export
# Define unit names renaming rules
renaming_unit_rules = {'VALUE_Bq/m³': 1, #'Bq/m3'
                       'VALUE_Bq/kg': 3 } #'Bq/kg'

In [216]:
tfm.dfs['seawater']['FILT'].unique()

array([nan, 'N', 'F', 'n'], dtype=object)

In [226]:
#| export 
def get_filtered_lut():
    df = pd.read_excel(filtered_lut_path(), usecols=['name','id'])
    return df.set_index('name').to_dict()['id']

In [234]:
get_filtered_lut()

{'Not applicable': -1, 'Not available': 0, 'Yes': 1, 'No': 2}

In [237]:
renaming_rules = {'N': 'No',
                  'n': 'No',
                  'F': 'Yes'}

HERE NEED TO complete the lookup

In [249]:
# | export
class LookupFiltCB(Callback):
    "Remamp value type to MARIS format."
    def __init__(self ,
                 rules=renaming_rules,
                 fn_lut=get_filtered_lut
                 ):
        fc.store_attr()

    def __call__(self, tfm):
        lut = self.fn_lut()
        rules = self.rules
        for grp in tfm.dfs.keys():
            if "FILT" in tfm.dfs[grp].columns:
                # Fill values that are not in the renaming rules with 'Not Available'.
                tfm.dfs[grp].loc[~tfm.dfs[grp]["FILT"].isin(list(rules.keys())), "FILT"] = 'Not available'
                
                '''
                # Rename HELCOM format with MARIS format. 
                tfm.dfs[grp]['FILT'] = tfm.dfs[grp]['FILT'].apply(lambda x: rules[x] if x in list(rules.keys()))
                # Perform lookup
                tfm.dfs[grp]['FILT'] = tfm.dfs[grp]['FILT'].apply(lambda x: lut[x])
                '''
                

In [250]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            NormalizeUncUnitCB(),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),
                            RemampDataProviderSampleIdCB(),
                            LookupFiltCB()
                            ])

print(tfm()['seawater']['FILT'].unique())

['Not available' 'N' 'F' 'n']


In [202]:
vars = cdl_cfg()
vars['vars']['suffixes'].keys()

dict_keys(['uncertainty', 'detection_limit', 'volume', 'salinity', 'temperature', 'filtered', 'counting_method', 'sampling_method', 'preparation_method', 'unit'])

In [ ]:
data_provider_sample_id

In [204]:
vars = cdl_cfg()['vars']
vars['defaults'].keys()

dict_keys(['data_provider_sample_id', 'lon', 'lat', 'smp_depth', 'tot_depth', 'time', 'area', 'sample_notes', 'measurement_notes'])

***

### Rename columns

here do a lookup and set vars

In [203]:
tfm.dfs['seawater'].head()

,KEY,NUCLIDE,METHOD,< VALUE_Bq/m³,VALUE_Bq/m³,ERROR%_m³,DATE_OF_ENTRY_x,COUNTRY,LABORATORY,SEQUENCE,...,SDEPTH,SALIN,TTEMP,FILT,MORS_SUBBASIN,HELCOM_SUBBASIN,DATE_OF_ENTRY_y,time,unit,detection_limit
0,WKRIL2012003,cs137,NaN,NaN,5.3,0.089888,08/20/14 00:00:00,90,KRIL,2012003,...,0.0,NaN,NaN,NaN,11,11,08/20/14 00:00:00,2012-05-23,1,1
1,WKRIL2012004,cs137,NaN,NaN,19.9,0.792020,08/20/14 00:00:00,90,KRIL,2012004,...,29.0,NaN,NaN,NaN,11,11,08/20/14 00:00:00,2012-05-23,1,1
2,WKRIL2012005,cs137,NaN,NaN,25.5,1.300500,08/20/14 00:00:00,90,KRIL,2012005,...,0.0,NaN,NaN,NaN,11,3,08/20/14 00:00:00,2012-06-17,1,1
3,WKRIL2012006,cs137,NaN,NaN,17.0,0.838100,08/20/14 00:00:00,90,KRIL,2012006,...,0.0,NaN,NaN,NaN,11,11,08/20/14 00:00:00,2012-05-24,1,1
4,WKRIL2012007,cs137,NaN,NaN,22.2,0.887112,08/20/14 00:00:00,90,KRIL,2012007,...,39.0,NaN,NaN,NaN,11,11,08/20/14 00:00:00,2012-05-24,1,1


In [194]:
tfm.dfs['seawater'].columns

Index(['KEY', 'NUCLIDE', 'METHOD', '< VALUE_Bq/m³', 'VALUE_Bq/m³', 'ERROR%_m³',
       'DATE_OF_ENTRY_x', 'COUNTRY', 'LABORATORY', 'SEQUENCE', 'DATE', 'YEAR',
       'MONTH', 'DAY', 'STATION', 'LATITUDE (ddmmmm)', 'LATITUDE (dddddd)',
       'LONGITUDE (ddmmmm)', 'LONGITUDE (dddddd)', 'TDEPTH', 'SDEPTH', 'SALIN',
       'TTEMP', 'FILT', 'MORS_SUBBASIN', 'HELCOM_SUBBASIN', 'DATE_OF_ENTRY_y',
       'time', 'unit', 'detection_limit'],
      dtype='object')

In [192]:
tfm.dfs['seawater']['METHOD'].unique()

array([nan, 'CLOR02', 'CLOR04', 'CLOR07', '2', '5', '1', '4', 'ERPC01',
       'IMGW01', 'IMGW02', 'JORC07', 'JORC05', 'JORC02', 'KRIL02', '0',
       'RISO02', 'RISO03', 'RISO04', '24', '49', '25', '46', '13', '11',
       'STUK01', 'STUK04', 'STUK02', 'STUK03', '3', '6', 'STUK05',
       'LEPA05', 'LEPA02', 'LVEA02', 'LVEA04', 'LEPA07', 'IMGW03',
       'SSSIxx', 'SSSIxy', 'EBRS01', 'STUK06', 'DHIG03', 'DHIG02',
       'DHIG06', 'SSSI01', 'RISO07', 'DHIG04', 'DHIG01', 'DHIG3',
       'LEPA08', 'SSSM01', 'SSSM02', 'SSSM46', 'SSM46', 'SSM45', 'LEPA09',
       'DHIG2', 'DHIG5a'], dtype=object)

In [ ]:
sampling_method

In [193]:
vars

{'defaults': {'data_provider_sample_id': {'name': 'data_provider_sample_id',
   'dtype': 'S1',
   'attrs': {'long_name': ' Data provider sample id',
    'standard_name': 'data_provider_sample_id'}},
  'lon': {'name': 'lon',
   'dtype': 'f4',
   'attrs': {'long_name': 'Measurement longitude',
    'standard_name': 'longitude',
    'units': 'degrees_north',
    'axis': 'Y',
    '_CoordinateAxisType': 'Lon'}},
  'lat': {'name': 'lat',
   'dtype': 'f4',
   'attrs': {'long_name': 'Measurement latitude',
    'standard_name': 'latitude',
    'units': 'degrees_east',
    'axis': 'X',
    '_CoordinateAxisType': 'Lat'}},
  'smp_depth': {'name': 'smp_depth',
   'dtype': 'f4',
   'attrs': {'long_name': 'Sample depth below seal level',
    'standard_name': 'sample_depth_below_sea_floor',
    'units': 'm',
    'axis': 'Z'}},
  'tot_depth': {'name': 'tot_depth',
   'dtype': 'f4',
   'attrs': {'long_name': 'Total depth below seal level',
    'standard_name': 'total_depth_below_sea_floor',
    'units': 

In [190]:
tfm.dfs['seawater'].columns

Index(['KEY', 'NUCLIDE', 'METHOD', '< VALUE_Bq/m³', 'VALUE_Bq/m³', 'ERROR%_m³',
       'DATE_OF_ENTRY_x', 'COUNTRY', 'LABORATORY', 'SEQUENCE', 'DATE', 'YEAR',
       'MONTH', 'DAY', 'STATION', 'LATITUDE (ddmmmm)', 'LATITUDE (dddddd)',
       'LONGITUDE (ddmmmm)', 'LONGITUDE (dddddd)', 'TDEPTH', 'SDEPTH', 'SALIN',
       'TTEMP', 'FILT', 'MORS_SUBBASIN', 'HELCOM_SUBBASIN', 'DATE_OF_ENTRY_y',
       'time', 'unit', 'detection_limit'],
      dtype='object')

In [177]:
vars = cdl_cfg()['vars']
vars['defaults'].keys()

dict_keys(['data_provider_sample_id', 'lon', 'lat', 'smp_depth', 'tot_depth', 'time', 'area', 'sample_notes', 'measurement_notes'])

In [179]:
vars.keys()

dict_keys(['defaults', 'bio', 'sed', 'suffixes'])

In [176]:
#vars['suffixes']

In [ ]:
data_provider_sample_id

In [185]:
cdl_cfg()['vars'].keys()

dict_keys(['defaults', 'bio', 'sed', 'suffixes'])

In [187]:
cdl_cfg()['vars']['defaults'].keys()

dict_keys(['data_provider_sample_id', 'lon', 'lat', 'smp_depth', 'tot_depth', 'time', 'area', 'sample_notes', 'measurement_notes'])

In [ ]:
#| export
# Define columns of interest (keys) and renaming rules (values).+
def get_renaming_rules():
    vars = cdl_cfg()['vars']
    return {('seawater','biota', 'sediment') : {    
                                                        ## DEFAULT
                                                        'LATITUDE (dddddd)' : vars['defaults']['lat']['name'] ,
                                                        'LONGITUDE (dddddd)' : vars['defaults']['lon']['name'] ,
                                                        'time': vars['defaults']['time']['name'],
                                                        'NUCLIDE': 'nuclide',
                                                        'unit': vars['suffixes']['unit']['name'],
                                                        '_unc' : 'Uncertainty',
                                                        #'Quality flag' : 'Quality flag'
                                                        #'Station ID' : 'Station ID '
                                                        #'Sample ID' : 'Sample ID'
                                                        #'Profile or transect ID' : 'Profile or transect ID'
                                                        #'Sampling method' : 'Sampling method'
                                                        #'Preparation method' : 'Preparation method'
                                                        #'Counting method' : 'Counting method'
                                                        #'Sample notes' : 'Sample notes'
                                                        #'Measurement notes' : 'Measurement notes'
                                                    },
                  ('seawater',) : {
                                ## SEAWATER
                                'VALUE_Bq/m³': 'value',
                                'ERROR%_m³': vars['suffixes']['uncertainty']['name'],
                                'TDEPTH': vars['defaults']['tot_depth']['name'],
                                'SDEPTH': vars['defaults']['smp_depth']['name'],
                                'SALIN' : vars['defaults']['salinity']['name'],
                                'TTEMP' :vars['defaults']['temperature']['name'],
                                #'smp_depth': 'Sampling depth',
                                #'Salinity' : 'Salinity',
                                #'Temperature' : 'Temperature',
                                #'Filtered' : 'Filtered',
                                #'Mesh size' : 'Mesh size',
                                #'Total depth' : 'Total depth'
                                },
                  ('biota',) : { 
                                ## BIOTA
                                'VALUE_Bq/kg': 'value',
                                'ERROR%': vars['suffixes']['uncertainty']['name'],
                                'SDEPTH': vars['defaults']['smp_depth']['name'],

                                
                                'Species' : 'Species',
                                'Body part' : 'Body part'
        'SDEPTH': vars['defaults']['smp_depth']['name'],
                                },
                  ('sediment',) : {
                                ## SEDIMENT
                                'VALUE_Bq/kg': 'value',
                                'ERROR%_kg': vars['suffixes']['uncertainty']['name'],
                                'TDEPTH': vars['defaults']['tot_depth']['name'],

                                #'Top' : 'Top',
                                #'Bottom' : 'Bottom',
                                #'Sediment type' : 'Sediment type'
                                }
                    }

In [ ]:
#| export
# Define columns of interest by sample type
coi_grp = {'seawater': ['NUCLIDE', 'VALUE_Bq/m³', 'ERROR%_m³', 'time',
                        'TDEPTH', 'SDEPTH', 'LATITUDE (dddddd)', 'LONGITUDE (dddddd)','unit', 'SALIN', 'TTEMP'],
           'sediment': ['NUCLIDE', 'VALUE_Bq/kg', 'ERROR%_kg', 'time',
                        'TDEPTH', 'LATITUDE (dddddd)', 'LONGITUDE (dddddd)',
                        'sed_type','unit'],
           'biota': ['NUCLIDE', 'VALUE_Bq/kg', 'ERROR%', 'time',
                     'SDEPTH', 'LATITUDE ddmmmm', 'LONGITUDE ddmmmm',
                     'species', 'body_part','unit', 'bio_group']}

In [ ]:
#| export
def get_renaming_rules():
    vars = cdl_cfg()['vars']
    # Define column names renaming rules
    return {
        'NUCLIDE': 'nuclide',
        'VALUE_Bq/m³': 'value',
        'VALUE_Bq/kg': 'value',
        'ERROR%_m³': vars['suffixes']['uncertainty']['name'],
        'ERROR%_kg': vars['suffixes']['uncertainty']['name'],
        'ERROR%': vars['suffixes']['uncertainty']['name'],
        'SDEPTH': vars['defaults']['smp_depth']['name'],
        'TDEPTH': vars['defaults']['tot_depth']['name'],
        'LATITUDE (dddddd)': vars['defaults']['lat']['name'],
        'LATITUDE ddmmmm': vars['defaults']['lat']['name'],
        'LONGITUDE (dddddd)': vars['defaults']['lon']['name'],
        'LONGITUDE ddmmmm': vars['defaults']['lon']['name'],
        'SALIN': vars['suffixes']['salinity']['name'],
        'TTEMP': vars['suffixes']['temperature']['name'],
        'unit': vars['suffixes']['unit']['name']
    }

In [ ]:
#| export
# Define column names renaming rules
#renaming_rules = {
#    'NUCLIDE': 'nuclide',
#    'VALUE_Bq/m³': 'value',
#    'VALUE_Bq/kg': 'value',
#    'ERROR%_m³': vars['suffixes']['uncertainty']['name'],
#    'ERROR%_kg': vars['suffixes']['uncertainty']['name'],
#    #'ERROR%': 'vars['suffixes']['uncertainty']['name'],
#    'SDEPTH': vars['defaults']['depth']['name'],
#    'LATITUDE (dddddd)': vars['defaults']['lat']['name'],
#    'LATITUDE ddmmmm': vars['defaults']['lat']['name'],
#    'LONGITUDE (dddddd)': vars['defaults']['lon']['name'],
#    'LONGITUDE ddmmmm': vars['defaults']['lon']['name'],
#    'SALIN': vars['suffixes']['salinity']['name'],
#    'TTEMP': vars['suffixes']['temperature']['name'],
#}


In [ ]:
#| export
class RenameColumnCB(Callback):
    def __init__(self,
                 coi,
                 fn_renaming_rules,
                ):
        fc.store_attr()

    def __call__(self, tfm):
        for k in tfm.dfs.keys():
            # Select cols of interest
            tfm.dfs[k] = tfm.dfs[k].loc[:, self.coi[k]]

            # Rename cols
            tfm.dfs[k].rename(columns=self.fn_renaming_rules(), inplace=True)

NameError: name 'Callback' is not defined

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart),
                            LookupSedimentCB(get_maris_sediments),
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupUnitCB(),
                            RenameColumnCB(coi_grp, get_renaming_rules)
                            ])

print(tfm()['seawater'].head())

NameError: name 'load_data' is not defined

### Reshape: long to wide

In [ ]:
#| export
class ReshapeLongToWide(Callback):
    "Convert data from long to wide with renamed columns."
    def __init__(self, columns='nuclide', values=['value']):
        fc.store_attr()
        # Retrieve all possible derived vars (e.g 'unc', 'dl', ...) from configs
        self.derived_cols = [value['name'] for value in cdl_cfg()['vars']['suffixes'].values()]
    
    def renamed_cols(self, cols):
        "Flatten columns name"
        return [inner if outer == "value" else f'{inner}{outer}'
                if inner else outer
                for outer, inner in cols]

    def pivot(self, df):
        # Among all possible 'derived cols' select the ones present in df
        derived_coi = [col for col in self.derived_cols if col in df.columns]
        
        df.reset_index(names='sample', inplace=True)
        
        idx = list(set(df.columns) - set([self.columns] + derived_coi + self.values))
        return df.pivot_table(index=idx,
                              columns=self.columns,
                              values=self.values + derived_coi,
                              fill_value=np.nan,
                              aggfunc=lambda x: x
                              ).reset_index()

    def __call__(self, tfm):
        for k in tfm.dfs.keys():
            tfm.dfs[k] = self.pivot(tfm.dfs[k])
            tfm.dfs[k].columns = self.renamed_cols(tfm.dfs[k].columns)

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart),
                            LookupSedimentCB(get_maris_sediments),
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupUnitCB(),
                            RenameColumnCB(coi_grp, get_renaming_rules),
                            ReshapeLongToWide()
                            ])

print(tfm()['biota'].head())

     lon       time  species  body_part  bio_group  sample    lat  smp_depth  \
0   9.41 2011-12-11       50         52          4     192  54.31        2.0   
1   9.41 2011-12-11       50         52          4     193  54.31        2.0   
2   9.41 2011-12-11       50         52          4     194  54.31        2.0   
3   9.41 2011-12-11       50         52          4     195  54.31        2.0   
4  10.00 2011-12-13       99         52          4     184  54.45        4.0   

   ac228_unc  ag108m_unc  ...  sr89  sr90  tc99  te129m  th228  th232  tl208  \
0        NaN         NaN  ...   NaN   NaN   NaN     NaN    NaN    NaN    NaN   
1        NaN         NaN  ...   NaN   NaN   NaN     NaN    NaN    NaN    NaN   
2        NaN         NaN  ...   NaN   NaN   NaN     NaN    NaN    NaN    NaN   
3        NaN         NaN  ...   NaN   NaN   NaN     NaN    NaN    NaN    NaN   
4        NaN         NaN  ...   NaN   NaN   NaN     NaN    NaN    NaN    NaN   

   u235  zn65  zr95  
0   NaN   NaN   

### Encode time (seconds since ...)

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart),
                            LookupSedimentCB(get_maris_sediments),
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupUnitCB(),
                            RenameColumnCB(coi_grp, get_renaming_rules),
                            ReshapeLongToWide(),
                            EncodeTimeCB(cfg())
                            ])

print(tfm()['biota'].head())

     lon        time  species  body_part  bio_group  sample    lat  smp_depth  \
0   9.41  1323561600       50         52          4     192  54.31        2.0   
1   9.41  1323561600       50         52          4     193  54.31        2.0   
2   9.41  1323561600       50         52          4     194  54.31        2.0   
3   9.41  1323561600       50         52          4     195  54.31        2.0   
4  10.00  1323734400       99         52          4     184  54.45        4.0   

   ac228_unc  ag108m_unc  ...  sr89  sr90  tc99  te129m  th228  th232  tl208  \
0        NaN         NaN  ...   NaN   NaN   NaN     NaN    NaN    NaN    NaN   
1        NaN         NaN  ...   NaN   NaN   NaN     NaN    NaN    NaN    NaN   
2        NaN         NaN  ...   NaN   NaN   NaN     NaN    NaN    NaN    NaN   
3        NaN         NaN  ...   NaN   NaN   NaN     NaN    NaN    NaN    NaN   
4        NaN         NaN  ...   NaN   NaN   NaN     NaN    NaN    NaN    NaN   

   u235  zn65  zr95  
0   NaN   

### Sanitize coordinates

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart),
                            LookupSedimentCB(get_maris_sediments),
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupUnitCB(),
                            RenameColumnCB(coi_grp, get_renaming_rules),
                            ReshapeLongToWide(),
                            EncodeTimeCB(cfg()),
                            SanitizeLonLatCB()
                            ])

print(tfm()['biota'].head())

     lon        time  species  body_part  bio_group  sample    lat  smp_depth  \
0   9.41  1323561600       50         52          4     192  54.31        2.0   
1   9.41  1323561600       50         52          4     193  54.31        2.0   
2   9.41  1323561600       50         52          4     194  54.31        2.0   
3   9.41  1323561600       50         52          4     195  54.31        2.0   
4  10.00  1323734400       99         52          4     184  54.45        4.0   

   ac228_unc  ag108m_unc  ...  sr89  sr90  tc99  te129m  th228  th232  tl208  \
0        NaN         NaN  ...   NaN   NaN   NaN     NaN    NaN    NaN    NaN   
1        NaN         NaN  ...   NaN   NaN   NaN     NaN    NaN    NaN    NaN   
2        NaN         NaN  ...   NaN   NaN   NaN     NaN    NaN    NaN    NaN   
3        NaN         NaN  ...   NaN   NaN   NaN     NaN    NaN    NaN    NaN   
4        NaN         NaN  ...   NaN   NaN   NaN     NaN    NaN    NaN    NaN   

   u235  zn65  zr95  
0   NaN   

## NetCDF encoder

### Example change logs

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart),
                            LookupSedimentCB(get_maris_sediments),
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupUnitCB(),
                            RenameColumnCB(coi_grp, get_renaming_rules),
                            ReshapeLongToWide(),
                            EncodeTimeCB(cfg()),
                            SanitizeLonLatCB()
                            ])

# Transform
tfm()

# Check transformation logs
tfm.logs

['Convert nuclide names to lowercase & strip any trailing space(s)',
 'Remap to MARIS radionuclide names.',
 '\n    Biota species remapped to MARIS db:\n        CARD EDU: Cerastoderma edule\n        LAMI SAC: Saccharina latissima\n        PSET MAX: Scophthalmus maximus\n        STIZ LUC: Sander luciopercas\n    ',
 "\n    Update bodypart id based on MARIS dbo_bodypar.xlsx:\n        - 3: 'Whole animal eviscerated without head',\n        - 12: 'Viscera',\n        - 8: 'Skin'\n    ",
 "\n    Update sediment id  based on MARIS dbo_sedtype.xlsx\n        -99: '(Not available)'\n        - na: '(Not available)'\n        - 56: '(Not available)'\n        - 73: '(Not available)'\n    ",
 '\n    Update biogroup id  based on MARIS dbo_species.xlsx\n    ',
 'Convert data from long to wide with renamed columns.',
 'Encode time as `int` representing seconds since xxx',
 'Drop row when both longitude & latitude equal 0. Drop unrealistic longitude & latitude values. Convert longitude & latitude `,` sepa

### Feed global attributes

In [ ]:
#| export
kw = ['oceanography', 'Earth Science > Oceans > Ocean Chemistry> Radionuclides',
      'Earth Science > Human Dimensions > Environmental Impacts > Nuclear Radiation Exposure',
      'Earth Science > Oceans > Ocean Chemistry > Ocean Tracers, Earth Science > Oceans > Marine Sediments',
      'Earth Science > Oceans > Ocean Chemistry, Earth Science > Oceans > Sea Ice > Isotopes',
      'Earth Science > Oceans > Water Quality > Ocean Contaminants',
      'Earth Science > Biological Classification > Animals/Vertebrates > Fish',
      'Earth Science > Biosphere > Ecosystems > Marine Ecosystems',
      'Earth Science > Biological Classification > Animals/Invertebrates > Mollusks',
      'Earth Science > Biological Classification > Animals/Invertebrates > Arthropods > Crustaceans',
      'Earth Science > Biological Classification > Plants > Macroalgae (Seaweeds)']


In [ ]:
#| export
def get_attrs(tfm, zotero_key, kw=kw):
    return GlobAttrsFeeder(tfm.dfs, cbs=[
        BboxCB(),
        DepthRangeCB(),
        TimeRangeCB(cfg()),
        ZoteroCB(zotero_key, cfg=cfg()),
        KeyValuePairCB('keywords', ', '.join(kw)),
        KeyValuePairCB('publisher_postprocess_logs', ', '.join(tfm.logs))
        ])()

In [ ]:
#|eval: false
get_attrs(tfm, zotero_key='26VMZZ2Q', kw=kw)

{'geospatial_lat_min': '31.1667',
 'geospatial_lat_max': '65.6347',
 'geospatial_lon_min': '9.41',
 'geospatial_lon_max': '53.458',
 'geospatial_bounds': 'POLYGON ((9.41 53.458, 31.1667 53.458, 31.1667 65.6347, 9.41 65.6347, 9.41 53.458))',
 'time_coverage_start': '1984-01-10T00:00:00',
 'time_coverage_end': '2021-12-06T00:00:00',
 'title': 'Environmental database - Helsinki Commission Monitoring of Radioactive Substances',
 'summary': 'MORS Environment database has been used to collate data resulting from monitoring of environmental radioactivity in the Baltic Sea based on HELCOM Recommendation 26/3.\n\nThe database is structured according to HELCOM Guidelines on Monitoring of Radioactive Substances (https://www.helcom.fi/wp-content/uploads/2019/08/Guidelines-for-Monitoring-of-Radioactive-Substances.pdf), which specifies reporting format, database structure, data types and obligatory parameters used for reporting data under Recommendation 26/3.\n\nThe database is updated and quality a

In [ ]:
#| export
def enums_xtra(tfm, vars):
    "Retrieve a subset of the lengthy enum as 'species_t' for instance"
    enums = Enums(lut_src_dir=lut_path(), cdl_enums=cdl_cfg()['enums'])
    xtras = {}
    for var in vars:
        unique_vals = tfm.unique(var)
        if unique_vals.any():
            xtras[f'{var}_t'] = enums.filter(f'{var}_t', unique_vals)
    return xtras

### Encoding

In [ ]:
#| export
def encode(fname_in, fname_out, nc_tpl_path, **kwargs):
    dfs = load_data(fname_in)
    tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                                RemapRdnNameCB(),
                                ParseTimeCB(),
                                LookupBiotaSpeciesCB(get_maris_species),
                                LookupBiotaBodyPartCB(get_maris_bodypart),
                                LookupSedimentCB(get_maris_sediments),
                                LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                                LookupUnitCB(),
                                RenameColumnCB(coi_grp, get_renaming_rules),
                                ReshapeLongToWide(),
                                EncodeTimeCB(cfg()),
                                SanitizeLonLatCB()
                                ])
    tfm()
    encoder = NetCDFEncoder(tfm.dfs, 
                            src_fname=nc_tpl_path,
                            dest_fname=fname_out, 
                            global_attrs=get_attrs(tfm, zotero_key='26VMZZ2Q', kw=kw),
                            verbose=kwargs.get('verbose', False),
                            enums_xtra=enums_xtra(tfm, vars=['species', 'body_part'])
                           )
    encoder.encode()

In [ ]:
#|eval: false
encode(fname_in, fname_out, nc_tpl_path(), verbose=False)

## TODO

TODO : METHOD, FILT (seawater)

TODO : Do we want to include laboratory code in NetCDF?